# 京东8.19笔试题
## 参考面经：
https://www.nowcoder.com/discuss/522382154890764288?sourceSSR=search

https://www.nowcoder.com/discuss/522399257547055104?sourceSSR=search

https://www.nowcoder.com/feed/main/detail/1bf0e809aaa04156b7b81559a0234fb1?sourceSSR=search

https://www.nowcoder.com/discuss/522381734340460544?sourceSSR=post

## 题目
编程题
1. 夹吃棋，棋会被左右两侧和上下两侧的异色棋吃掉，计算谁赢，模拟
2. 给m个症状和n个药，药有治疗的症状和产生的副作用，计算吃第i个药后的症状，模拟
3. 皇后走棋盘，一次可以向右、向下、向右下走，棋盘有障碍物。求从左上角到右下角的最少步数。动态规划，三维数组，第三维存储三个方向的最少步数。


1. 夹吃棋，棋会被左右两侧和上下两侧的异色棋吃掉，计算谁赢，模拟

    *代表黑色，o代表红色，.代表空白。只有一方吃了另一方（不是互相吃）算赢，否则都是平局

In [12]:
case1 = """
*oo
o.*
**o
"""

def valid(x, y):
    return (x >= 0 and x < 3) and (y >= 0 and y < 3)

def judge(chess):
    red = 0
    blc = 0
    for i in range(3):
        for j in range(3):
            if chess[i][j] == '*':
                if valid(i-1, j) and valid(i+1, j) and chess[i-1][j] == 'o' and chess[i+1][j] == 'o':
                    red += 1
                if valid(i, j-1) and valid(i, j+1) and chess[i][j-1] == 'o' and chess[i][j+1] == 'o':
                    red += 1
            elif chess[i][j] == 'o':
                if valid(i-1, j) and valid(i+1, j) and chess[i-1][j] == '*' and chess[i+1][j] == '*':
                    blc += 1
                if valid(i, j-1) and valid(i, j+1) and chess[i][j-1] == '*' and chess[i][j+1] == '*':
                    blc += 1
    if (red == 0 and blc == 0) or (red and blc):
        return 'draw'
    elif red == 0:
        return 'black'
    else:
        return 'red'

chess = []
for i in case1.strip().split('\n'):
    chess.append(list(i))
print(chess)
print(judge(chess))

[['*', 'o', 'o'], ['o', '.', '*'], ['*', '*', 'o']]
draw


2. 给m个症状和n个药，药有治疗的症状和产生的副作用，计算吃第i个药后的症状，模拟

    这个太简单了，跳过了，因为确实是模拟，上一题还有循环判断，这题纯粹顺序模拟

3. 皇后走棋盘，一次性可以向右、向下、向右下走`k`距离但只算一步，其中右下是指(x, y)=>(x+k, y+k)，棋盘有障碍物`*`。求从左上角到右下角的最少步数。
   
   动态规划，三维数组，第三维存储三个方向的最少步数。

先复习一下最朴素的dfs解法，在考场上会超时但是有一定的ac率的。

In [17]:
case3 = """
....
.*.*
*.*.
....
"""

chess = []
for i in case3.strip().split('\n'):
    chess.append(list(i))

def minSteps(chess):
    moves = [(1, 0, '↓'), (1, 1, '↘'), (0, 1, '→')]
    n, m = len(chess), len(chess[0])
    paths = []
    ans = float('inf')
    def valid(x, y, i, j):
        return x+i>=0 and x+i<n and y+j>=0 and y+j<m and chess[x+i][y+j]=='.' 

    def dfs(x, y):
        nonlocal ans
        if x == n-1 and y == m-1:
            cur = 1
            for i in range(1, len(paths)):
                if paths[i] != paths[i-1]:
                    cur += 1
            if cur < ans:
                for i in chess:print(' '.join(i))
                print()
                ans = min(ans, cur)
        for x_forward, y_forward, direction in moves:
            if valid(x, y, x_forward, y_forward):
                paths.append((x_forward, y_forward))
                chess[x+x_forward][y+y_forward] = direction
                dfs(x+x_forward, y+y_forward)
                paths.pop(-1)
                chess[x+x_forward][y+y_forward] = '.'
    dfs(0, 0)
    return ans

print(minSteps(chess))

. . . .
↓ * . *
* ↘ * .
. ↓ → →

. . . .
↓ * . *
* ↘ * .
. . ↘ →

3


上面这个例子失败的原因是时间复杂度太高了，实际上可以用空间换时间，使用BFS+动态规划的方式解决：

+ dp[i][j]代表棋子到达点`(i, j)`所花费的最小步数
+ 对与任意一个出发点`(x, y)`，它在一次BFS遍历中应该能一步到达当前所有横行、竖行和斜行，所以dp[x+k][y] = dp[x][y] + 1，其他两个方向同理，这是本题的关键
+ BFS过程中不会设置visited数组记录访问过的节点，因为访问过的节点dp值可能不是最小的还会更新，所以每当dp[i][j]发生了更新，将其重新加入队列，让它更新其他位置，直到最后没有节点有更新，此时队列为空

In [2]:
case3 = """
....
.*.*
*.*.
....
"""
chess = []
for i in case3.strip().split('\n'):
    chess.append(list(i))
# for i in chess:print(i)

def minSteps_dp(chess):
    n, m = len(chess), len(chess[0])
    dp = [[float('inf') for _ in range(m)] for _ in range(n)]
    def valid(x, y, i, j):
        return x+i>=0 and x+i<n and y+j>=0 and y+j<m and chess[x+i][y+j]=='.' 
    dp[0][0] = 0
    q = [(0, 0)]
    while q:
        x, y = q.pop(0)
        k = 1
        while valid(x, y, k, 0):
            if dp[x+k][y] > dp[x][y]+1:
                dp[x+k][y] = dp[x][y]+1
                q.append((x+k, y))
            k += 1
        k = 1
        while valid(x, y, 0, k):
            if dp[x][y+k] > dp[x][y]+1:
                dp[x][y+k] = dp[x][y]+1
                q.append((x, y+k))
            k += 1
        k = 1
        while valid(x, y, k, k):
            if dp[x+k][y+k] >= dp[x][y]+1:
                dp[x+k][y+k] = dp[x][y]+1
                q.append((x+k, y+k))
            k += 1
    return dp[-1][-1]

print(minSteps_dp(chess))

3


补充一题 09.10 JD Star：

小红定义一个长度为 m 的数组 b 的权值为：

$1*b_{1}+2*b_{2}+3*b_{3}+...+m*b_{m}$

现在小红有一个长度为n的数组a，她想知道所有子数组的权值和是多少?答案对10^9+7取模

输入示例：

3

1， 2， 3

输出：

33

子数组[1]的权值是1

子数组[1,2]的权值是5

子数组[1,2,3]的权值是14

子数组[2]的权值是2

子数组[2,3]的权值是8

子数组[3]的权值是3

In [9]:
def solve(n, A):
    dp = [['']*(n+1) for _ in range(n+1)]
    for j in range(n+1):
        dp[0][j] = sum([(i+1)*A[i] for i in range(j)])
    # dp[i][j]表示i,j的权重合
    for i in range(1, n+1):
        for j in range(i, n+1):
            if i == j:
                dp[i][j] = A[i-1]
            elif i == 1:
                dp[i][j] = dp[0][j]
            else:
                dp[i][j] = dp[i-1][j] - sum(A[i-2:j]) # 这里的区间和计算可以用前缀和优化
    for i in dp:print(i)
    ans = 0
    for i in range(1, n+1):
        for j in range(i, n+1):
            ans += dp[i][j]
    return ans


A = [1, 2, 3]
n = 3
print(solve(3, A))

[0, 1, 5, 14]
['', 1, 5, 14]
['', '', 2, 8]
['', '', '', 3]
33
